In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
# Model
m = gp.Model("district")
m

Restricted license - for non-production use only - expires 2025-11-24


<gurobi.Model Continuous instance district: 0 constrs, 0 vars, No parameter changes>

In [4]:
state = 'RI'
path = '../data/{}/counties/graph'.format(state)
population = '{}/{}.population'.format(path, state)
dimacs = '{}/{}.dimacs'.format(path, state)

with open('../Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts = int(parts[1])


with open(population, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)

num_districts


2

In [5]:
## Reading distances from file
filename = '{}/{}_distances.csv'.format(path, state)
d = np.genfromtxt(filename, delimiter=',', skip_header=1)
d = d[:, 1:]

## Reading populations from file
with open(population, 'r') as file:
    # Read the total population from the first line
    total_population_line = next(file).strip()  # Removes newline character at the end
    total_population = int(total_population_line.split('=')[1].strip())
    
    # Initialize a dictionary to store the rest of the data
    p = {}
    
    # Iterate over the remaining lines
    for line in file:
        # Each line is expected to have two values, separated by a space
        key, value = line.strip().split()  # Split by whitespace
        p[int(key)] = int(value)  # Convert both to integers and store in the dictionary

## Reading neighbors from file
neighbors = set()
with open(dimacs, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('e'):
            _, node1, node2 = line.split()
            node1 = int(node1)
            node2 = int(node2)
            if (node1, node2) not in neighbors and (node2, node1) not in neighbors:
                neighbors.add((node1, node2))

list(neighbors)

[(0, 1), (2, 4), (1, 2), (3, 4), (0, 3), (2, 3), (1, 3)]

In [42]:
# Set decision variable x for each node i whether it is in district j
x = {}
for i in range(num_counties):
    for j in range(num_districts):
        x[i, j] = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

## Set decision variable y for each edge in neighbors, between nodes i and k, for each district j
y = {}
for i, k in neighbors:
    for j in range(num_districts):
        y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# for i in range(num_counties):
#     for k in range(num_counties):
#         for j in range(num_districts):
#             y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# Set slack variable
slack = {}
for j in range(num_districts):
    slack[j] = m.addVar(vtype=gp.GRB.CONTINUOUS, name="slack")




alpha = 1  # Weight of the balanced weight term
y


{(0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Var *Awaiting Model Update*>}

In [50]:
# Set objective function
m.setObjective(gp.quicksum(y[i, k, j] * d[i, k] for i, k in neighbors for j in range(num_districts)) \
             + alpha * (gp.quicksum(slack[j] for j in range(num_districts))), gp.GRB.MINIMIZE)


In [52]:
## Add constraints

# Each edge in neighbors is assigned to exactly one district
for i, k in neighbors:
    for j in range(num_districts):
        m.addConstr(x[i, j] + x[k, j] >= 2 * y[i, k, j] )

# Add population constraint with slack variable
# for j in range(num_districts):
#     m.addConstr(gp.quicksum(x[i, j] * p[i] for i in range(num_counties)) >= ((total_population / num_districts) - slack[j]))
#     m.addConstr(gp.quicksum(x[i, j] * p[i] for i in range(num_counties)) <= ((total_population / num_districts) + slack[j]))

# Each county is assigned to exactly one district
for i in range(num_counties):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_districts)) == 1)



In [54]:
# Write LP formulation to a file
m.write("districts.lp")

# Read the LP formulation from the file
with open("districts.lp", "r") as f:
    lp_formulation = f.read()

# Print LP formulation
print("LP Formulation:")
print(lp_formulation)

LP Formulation:
\ Model district
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0 x_0_0 + 0 x_0_1 + 0 x_1_0 + 0 x_1_1 + 0 x_2_0 + 0 x_2_1 + 0 x_3_0
   + 0 x_3_1 + 0 x_4_0 + 0 x_4_1 + 0 y_0_1_0 + 0 y_0_1_1 + 0 y_2_4_0
   + 0 y_2_4_1 + 0 y_1_2_0 + 0 y_1_2_1 + 0 y_3_4_0 + 0 y_3_4_1 + 0 y_0_3_0
   + 0 y_0_3_1 + 0 y_2_3_0 + 0 y_2_3_1 + 0 y_1_3_0 + 0 y_1_3_1 + 0 x_0_0
   + 0 x_0_1 + 0 x_1_0 + 0 x_1_1 + 0 x_2_0 + 0 x_2_1 + 0 x_3_0 + 0 x_3_1
   + 0 x_4_0 + 0 x_4_1 + 0 y_0_1_0 + 0 y_0_1_1 + 0 y_2_4_0 + 0 y_2_4_1
   + 0 y_1_2_0 + 0 y_1_2_1 + 0 y_3_4_0 + 0 y_3_4_1 + 0 y_0_3_0 + 0 y_0_3_1
   + 0 y_2_3_0 + 0 y_2_3_1 + 0 y_1_3_0 + 0 y_1_3_1 + 31 y_0_1_0
   + 31 y_0_1_1 + 31 y_2_4_0 + 31 y_2_4_1 + 23 y_1_2_0 + 23 y_1_2_1
   + 31 y_3_4_0 + 31 y_3_4_1 + 22 y_0_3_0 + 22 y_0_3_1 + 31 y_2_3_0
   + 31 y_2_3_1 + 25 y_1_3_0 + 25 y_1_3_1 + slack + slack
Subject To
 R0: x_0_0 + x_1_0 - 2 y_0_1_0 <= 0
 R1: x_0_1 + x_1_1 - 2 y_0_1_1 <= 0
 R2: x_2_0 + x_4_0 - 2 y_2_4_0 <

In [55]:
m.optimize()
# If model is infeasible, compute an Irreducible Inconsistent Subsystem (IIS)
if m.status == GRB.INFEASIBLE:
    m.computeIIS()
    m.write("model.ilp")
    print("IIS written to file 'model.ilp'")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 329 rows, 152 columns and 1044 nonzeros
Model fingerprint: 0x6eb6d013
Variable types: 8 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+05]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R148 by 548689.500000000

Presolve removed 86 rows and 80 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: App

In [26]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 157 rows, 26 columns and 508 nonzeros
Model fingerprint: 0x6a71ef0f
Variable types: 2 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+05]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R148 by 548689.500000000

Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [27]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nObjective Value: %g' % m.ObjVal)
        for i in range(num_counties):
            for j in range(num_districts):
                print("x_{}_{} = {}".format(i, j, x[i, j].X))
        for j in range(num_districts):
            print("slack_{} = {}".format(j, slack[j].X))
        for i,k in neighbors:
            for j in range(num_districts):
                print(f'y_{i}_{k}_{j} = {y[i, k, j].X}')
    else:
        print('No solution')

printSolution()

No solution
